In [1]:
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

In [46]:
# 라이브러리 선언
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("AI-SEARCH-POC")

LangSmith 추적을 시작합니다.
[프로젝트명]
AI-SEARCH-POC


In [3]:
# 임베딩(Embedding) 생성
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")  # 1536 차원

# DB 경로 설정
persist_directory = "../.db/faiss"

vectorstore = FAISS.load_local(
    persist_directory, 
    embeddings, 
    allow_dangerous_deserialization=True)

print(f"FAISS 인덱스 크기: {vectorstore.index.ntotal}개의 문서")

FAISS 인덱스 크기: 224361개의 문서


In [47]:
# OpenAI 객체를 생성합니다.
model = ChatOpenAI(temperature=0, model_name="gpt-4.1-mini")

In [48]:
# 원하는 데이터 구조를 정의합니다.
class Filter(BaseModel):
    SALE_PRC_GTE: int = Field(description="가격범위 최소값. 예를 들어 100만원 이상 120만원 이하이면, 1000000")
    SALE_PRC_LTE: int = Field(description="가격범위 최대값. 예를 들어 100만원 이상 120만원 이하이면, 1200000")
    BRND_NM: str = Field(description="브랜드명")
    ARTC_NM: str = Field(description="품목명")
    CATEGORY_NM: str = Field(description="""카테고리명. 다음 카테고리 중에 1개를 선택.
테마관
,TV·영상가전
,세탁기·건조기·의류관리기
,냉장고·주방가전
,청소기·생활가전
,에어컨·계절가전
,컴퓨터·노트북
,카메라
,음향가전
,게임기·타이틀
,휴대폰·스마트워치
,태블릿·이북리더기
,스마트홈
,건강가전
,뷰티·이미용가전
,가구·인테리어
,생활·주방용품
,문구·악기·공구
,레저·여행·헬스케어
,렌탈·구독
,안심케어
,방문컨설팅
,전문가 화상상담
,1인 가구를 위한 나노스퀘어""")
    FEATURES: str = Field(description="주요기능")

# {'SALE_PRC': {'$gte': 1000000, '$lte': 1200000}}

In [49]:
# 파서를 설정하고 프롬프트 템플릿에 지시사항을 주입합니다.
parser = JsonOutputParser(pydantic_object=Filter)
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"SALE_PRC_GTE": {"description": "가격범위 최소값. 예를 들어 100만원 이상 120만원 이하이면, 1000000", "title": "Sale Prc Gte", "type": "integer"}, "SALE_PRC_LTE": {"description": "가격범위 최대값. 예를 들어 100만원 이상 120만원 이하이면, 1200000", "title": "Sale Prc Lte", "type": "integer"}, "BRND_NM": {"description": "브랜드명", "title": "Brnd Nm", "type": "string"}, "ARTC_NM": {"description": "품목명", "title": "Artc Nm", "type": "string"}, "CATEGORY_NM": {"description": "카테고리명. 다음 카테고리 중에 1개를 선택.\n테마관\n,TV·영상가전\n,세탁기·건조기·의류관리기\n,냉장고·주방가전\n,청소기·생활가전\n,에어컨·계절가전\n,컴퓨터·노트북\n,카메

In [50]:

# 프롬프트를 생성합니다.
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 가전 전문 e커머스 검색 AI 어시스턴트 입니다. 사용자의 자연어 검색어를 분석하여, 그 의도에 맞는 최적의 검색 필터 조건을 완성해야 합니다."),
        ("user", "#Format: {format_instructions}\n\n#Question: {query}"),
    ]
)

prompt = prompt.partial(format_instructions=parser.get_format_instructions())

# 체인을 구성합니다.
chain = prompt | model | parser

In [81]:
def pretty_print(query, result):
    print(f"'{query}'와 유사한 상품 검색 결과:")
    print("-" * 60)

    if result:
        for i, (doc) in enumerate(result, 1):
            print(f"[결과 {i}]")
            print(f"{doc.page_content}")
            # print(f"메타데이터: {doc.metadata}")  # 모든 메타데이터 출력
            try:
                print(f"- 상품상태: {doc.metadata.get('GOODS_STAT_SCT_NM', '정보 없음')}")
                print(f"- 브랜드명: {doc.metadata.get('BRND_NM', '정보 없음')}")
                print(f"- 상품명: {doc.metadata.get('GOODS_NM', '정보 없음')}")
                print(f"- 품목: {doc.metadata.get('ARTC_NM', '정보 없음')}")
                print(f"- 카테고리: {doc.metadata.get('LGRP_NM', '정보 없음')}")
                print(f"- 판매가: {format(int(doc.metadata.get('SALE_PRC', 0)), ',')}원")
                # print(f"- 할인가: {format(int(doc.metadata.get('DSCNT_SALE_PRC', 0)), ',')}원")
                # print(f"- 최대혜택가: {format(int(doc.metadata.get('MAX_BENEFIT_PRICE', 0)), ',')}원")
                print(f"- 판매량: {format(int(doc.metadata.get('SALE_QTY', 0)), ',')}")
                print(f"- 세일즈유닛: {format(int(doc.metadata.get('SALES_UNIT', 0)), ',')}")
                # print(f"- 카드할인율: {doc.metadata.get('CARD_DC_RATE', '0')}%")
                # print(f"- 할인카드: {doc.metadata.get('CARD_DC_NAME_LIST', '정보 없음')}")
                print(f"- 주요 특징 및 기능:")
                feature_values = doc.metadata['OPT_VAL_DESC'].split(',')
                feature_titles = doc.metadata['OPT_DISP_NM'].split(',')
                for i, (title, value) in enumerate(zip(feature_titles, feature_values)):
                    print(f"  - {title} : {value}")
                    if i == 3: break
                print(f"🔗 상품보러가기 : https://www.e-himart.co.kr/app/goods/goodsDetail?goodsNo={doc.metadata.get('GOODS_NO', '정보 없음')}")
            except Exception as e:
                print(f"메타데이터 처리 중 오류 발생: {str(e)}")
            print("-" * 60)
    else:
        print("검색 결과가 없습니다.")
        print("다른 검색어로 다시 시도해보시거나, 검색어를 더 구체적으로 입력해주세요.")

In [5]:
def extract_documents_from_faiss(faiss_db):
    """FAISS DB에서 Document 객체들을 추출"""
    documents = []
    
    # docstore에서 모든 Document 객체 추출
    for doc_id in faiss_db.docstore._dict.keys():
        doc = faiss_db.docstore.search(doc_id)
        documents.append(doc)
    
    return documents

In [6]:
import re

def classify_query_type(query: str) -> str:
    """
    검색어가 자연어인지 키워드인지 간단한 규칙 기반으로 판단합니다.
    """
    query = query.strip()

    # 1. 길이 기반 판단
    words = query.split()
    num_words = len(words)
    if num_words > 5: # 5단어 초과면 자연어일 가능성 높음
        return "natural_language"

    # 2. 불용어 및 조사/어미 패턴 확인 (한국어 기준)
    # 간단한 불용어 리스트와 문장 종결 어미, 조사 패턴
    stopwords_korean = ["은", "는", "이", "가", "을", "를", "에", "에서", "와", "과", "로", "으로", "도", "만", "좀", "요", "입니다", "있나요", "해주세요", "추천해주세요", "어떤", "무엇"]
    
    for sw in stopwords_korean:
        if sw in query:
            # 불용어가 포함되어 있으면 자연어일 가능성 높음 (단, 짧은 쿼리 예외 처리 필요)
            if num_words > 2: # 2단어 초과 쿼리에 불용어 포함 시 자연어
                return "natural_language"

    # 3. 질문/명령형 어미 확인 (더 명확한 자연어 신호)
    if re.search(r'(주세요|해줘|추천|있나|있나요|떤가|떤가요|을까|을까요|뭔가요)\s*$', query):
        return "natural_language"
    
    # 4. 명확한 키워드 패턴 (예: 브랜드 + 모델명)
    # 실제 시스템에서는 상품명 DB와 비교하여 판단할 수 있습니다.
    # 여기서는 간단히 '브랜드 + 모델명' 패턴을 가정
    # if re.match(r'^(삼성|LG|애플|다이슨)\s+.*(TV|냉장고|폰|청소기|세탁기|에어컨)', query, re.IGNORECASE):
    #      if num_words <= 4: # 짧은 키워드 조합일 경우
    #          return "keyword"


    # 위 조건에 해당하지 않으면 기본적으로 키워드로 간주 (더 짧고 명료한 쿼리)
    return "keyword"

In [43]:
from kiwipiepy import Kiwi

kiwi = Kiwi()

def extract_noun_from_query(query):
    
    tokenized_query = kiwi.tokenize(query)

    print(f"{tokenized_query}")

    nng_forms = [token.form for token in tokenized_query if token.tag == 'NNG']
    return ','.join(nng_forms)

extract_noun_from_query("아기를 위한 가습기 추천")

[Token(form='아기', tag='NNG', start=0, len=2), Token(form='를', tag='JKO', start=2, len=1), Token(form='위하', tag='VV', start=4, len=2), Token(form='ᆫ', tag='ETM', start=5, len=1), Token(form='가습기', tag='NNG', start=7, len=3), Token(form='추천', tag='NNG', start=11, len=2)]


'아기,가습기,추천'

In [55]:
# 쿼리에 기반한 사용자 정의 필터를 반환합니다

import re

def price_filter_with_custom(query):
    """사용자 의도 분석을 통한 필터링"""

    filter_dict = {}

    # 가격 필터링을 위한 정규표현식 패턴
    # 1) N만원 이상 | N만원 부터
    price_pattern = r'(\d+)만원\s*(이상|부터)'
    price_match = re.search(price_pattern, query)
    
    if price_match:
        # SALE_PRC 키가 없으면 생성
        if "SALE_PRC" not in filter_dict:
            filter_dict["SALE_PRC"] = {}
        # 만원 단위를 원 단위로 변환 (예: 120만원 -> 1200000원)
        filter_dict["SALE_PRC"]["$gte"] = int(price_match.group(1)) * 10000
    
    # 2) N만원 이하 | N만원 까지
    price_pattern_lte = r'(\d+)만원\s*(이하|까지)'
    price_match_lte = re.search(price_pattern_lte, query)
    
    if price_match_lte:
        if "SALE_PRC" not in filter_dict:
            filter_dict["SALE_PRC"] = {}
        filter_dict["SALE_PRC"]["$lte"] = int(price_match_lte.group(1)) * 10000

    # 3) N만원 대
    price_pattern_range = r'(\d+)만원\s*대'
    price_match_range = re.search(price_pattern_range, query)
    
    if price_match_range:
        if "SALE_PRC" not in filter_dict:
            filter_dict["SALE_PRC"] = {}
        base_price = int(price_match_range.group(1)) * 10000
        filter_dict["SALE_PRC"]["$gte"] = base_price
        filter_dict["SALE_PRC"]["$lt"] = base_price * 2

    return filter_dict


In [79]:
# llm 이 분석한 의도 기반 필터를 반환합니다

def price_filter_with_llm(intent):
    filter_dict = {}
    # 가격
    if "SALE_PRC" not in filter_dict:
        filter_dict["SALE_PRC"] = {}
    filter_dict["SALE_PRC"]["$gte"] = intent["SALE_PRC_GTE"]
    filter_dict["SALE_PRC"]["$lte"] = intent["SALE_PRC_LTE"]
    return filter_dict

def brand_filter_with_llm(intent):
    filter_dict = {}
    # 브랜드
    filter_dict["BRND_NM"] = intent["BRND_NM"]
    return filter_dict

def artc_filter_with_llm(intent):
    filter_dict = {}
    # 품목
    filter_dict["ARTC_NM"] = intent["ARTC_NM"]
    return filter_dict

def category_filter_with_llm(intent):
    filter_dict = {}
    # 카테고리
    filter_dict["LGRP_NM"] = intent["CATEGORY_NM"]
    return filter_dict

def features_filter_with_llm(intent):
    filter_dict = {}
    # 주요 기능 및 특징
    filter_dict["FEATURES"] = intent["FEATURES"]
    return filter_dict

In [56]:
# 의도 기반 필터링 함수 선언
def intent_based_filtering(query, intent):
    """사용자 의도 분석을 통한 필터링"""

    filter_dict = {}

    # 가격 필터
    filter_dict.update(price_filter_with_custom(query) or price_filter_with_llm(intent))

    # 브랜드 필터
    filter_dict.update(brand_filter_with_llm(intent))

    # 카테고리 필터
    filter_dict.update(category_filter_with_llm(intent))

    # 품목 필터
    filter_dict.update(artc_filter_with_llm(intent))

    # 주요 기능 및 특징 필터
    filter_dict.update(features_filter_with_llm(intent))

    return filter_dict

In [78]:
from langchain_core.runnables import ConfigurableField

# faiss retriever를 초기화합니다.
faiss_retriever = vectorstore.as_retriever(search_kwargs={"k": 3}).configurable_fields(
    # search_type=ConfigurableField(
    #     id="search_type",
    #     name="Search Type",
    #     description="The search type to use",
    # ),
    search_kwargs=ConfigurableField(
        # 검색 매개변수의 고유 식별자를 설정
        id="search_kwargs",
        # 검색 매개변수의 이름을 설정
        name="Search Kwargs",
        # 검색 매개변수에 대한 설명을 작성
        description="The search kwargs to use",
    ),
)

# bm25 retriever를 초기화합니다.
documents = extract_documents_from_faiss(vectorstore)
bm25_retriever = BM25Retriever.from_documents(documents)
# bm25_retriever.k = 3

# 7대3 ensemble retriever를 초기화합니다.
bm25_faiss_73 = EnsembleRetriever(
    retrievers=[bm25_retriever, faiss_retriever],
    weights=[0.7, 0.3],
    search_kwargs={"k": 3}
)
a
# 3대7 ensemble retriever를 초기화합니다.
bm25_faiss_37 = EnsembleRetriever(
    retrievers=[bm25_retriever, faiss_retriever],
    weights=[0.3, 0.7],
    search_kwargs={"k": 3}
)

In [82]:

# 쿼리
query = "삼성 냉장고 추천"

# 필터
intent = chain.invoke({"query": query})
filter_dict = intent_based_filtering(query, intent)

print(filter_dict)

# config = {"configurable": {
#     "search_kwargs": {
#         "filter": {
#             'CATEGORY_NM': '냉장고·주방가전'
#             }
#         }
#     }
# }
# results = faiss_retriever.invoke(query, config=config)
results = faiss_retriever.invoke(query)

# # 쿼리타입에 따라 검색기를 선택합니다.
# query_type = classify_query_type(query)
# print(f"쿼리타입: {query_type}")

# if query_type == "keyword":
#     kiwi.tokenize(query)
#     results = bm25_faiss_73.invoke(query)
# elif query_type =="natural_language":
#     results = bm25_faiss_37.invoke(query)
# else:
#     results = faiss_retriever.invoke(query)

# # results = faiss_retriever.invoke(query)

pretty_print(query, results)

{'SALE_PRC': {'$gte': 0, '$lte': 100000000}, 'BRND_NM': '삼성', 'LGRP_NM': '냉장고·주방가전', 'ARTC_NM': '냉장고', 'FEATURES': ''}
'삼성 냉장고 추천'와 유사한 상품 검색 결과:
------------------------------------------------------------
[결과 1]
삼성전자 브랜드의 상품명 삼성 일반냉장고 298L RT31CG5624S9. 모델명: RT31CG5624S9-. 제품의 주요 특징은 품목: 일반형, 전체용량: 298L, 도어수: 2도어, e효율등급: 1등급, 소비전력(월): 27.3kWh, 크기(폭): 600mm, 색상: 실버 계열, 냉동실 용량: 67L, 냉장실 용량: 231L, 종류: 냉장고, 출시년도: 2023년, 도어 소재: 메탈 . 카테고리: 냉장고·주방가전. 가격 정보: 판매가 621,398원. 해시태그: #298L#RT31CG5624S9#냉장고#삼성. 에너지 효율 등급: 1등급. 2023년 5월에 출시되었으며 최근 한 달 동안 0개 판매됨.
- 상품상태: 정상상품
- 브랜드명: 삼성전자
- 상품명: 삼성 일반냉장고 298L RT31CG5624S9
- 품목: 일반냉장고
- 카테고리: 냉장고·주방가전
- 판매가: 621,398원
- 판매량: 0
- 세일즈유닛: 0
- 주요 특징 및 기능:
  - 품목 : 일반형
  - 전체용량 : 298L
  - 도어수 : 2도어
  - e효율등급 : 1등급
🔗 상품보러가기 : https://www.e-himart.co.kr/app/goods/goodsDetail?goodsNo=0038911686
------------------------------------------------------------
[결과 2]
삼성전자 브랜드의 상품명 양문형냉장고 RS84DB5002CW . 모델명: RS84DB5002CW. 제품의 주요 특징은 품목: 양문형, 전체용량: 852L, 도어수: 2도어, e효율